# Convertidor Recibos PDF a CSV

Este script extrae las tablas de los recibos de haberes de TASA ubicados en la carpeta `.\Recibos`, los unifica y lo exporta en formato CSV.

Los archivos deben nombrarse con el formato `Recibo AAAA-MM.pdf`.

### Extracción de datos con Tabula-Py

In [ ]:
import pandas as pd
import tabula
import re
from glob import glob


def to_num(column):
    """ Convierte el formato de las columnas para poder convertirlas a tipo numérico """
    column = str(column)
    column = re.sub('(\d*.\d*,\d{2})-','-\\1', column)
    column = column.replace('.','')
    column = column.replace(',','.')
    return pd.to_numeric(column, errors='coerce')
    

recibos_list = []

for file_name in glob('Recibos\*.pdf'):
    pdf = tabula.read_pdf(file_name, pages='all', multiple_tables=True, stream=True, pandas_options={'header': None},
                         area=(147.0,417.316,344.87,812.004),
                         columns=(449.3, 557.28, 595.4, 638.64, 697.68, 748.08, 811.44))
    
    df_partials = []
    
    for df_part in pdf:
        df_part.columns = ['codigo','concepto','unidad','valor','haberes_s_ret','haberes_c_ret','descuentos']
        df_part = df_part[df_part.codigo != 'Código'] # Elimina header de la página 1
        df_partials.append(df_part)
    
    # Concatena las paginas del recibo y resetea el índice
    df = pd.concat(df_partials)
    df.reset_index(inplace=True, drop=True)
    
    # Extrae período del nombre del archivo y lo anexa al dataframe
    df['periodo'] = re.search('(\d{4}-\d{2})', file_name).group(1) 
    

    # Convierte tipo de datos para campos numéricos:
    df['unidad'] = pd.to_numeric(df['unidad'].str.replace(',','.'), errors='coerce')
    df['valor'] = pd.to_numeric(df['valor'].str.replace(',','.'), errors='coerce')
    df['haberes_s_ret'] = df['haberes_s_ret'].apply(to_num)
    df['haberes_c_ret'] = df['haberes_c_ret'].apply(to_num)
    df['descuentos'] = df['descuentos'].apply(to_num)
          
    
    # Elimina filas importadas con subtotales (recibos viejos un poco más cortos)
    df = df[~df[['codigo','concepto']].isnull().all(1)]
    
    # Suma recibo completo a la lista
    recibos_list.append(df)

# Concatena todos los recibos
df_recibos = pd.concat(recibos_list)

#display(df_recibos)

### Exporta DataFrame a CSV

In [ ]:
df_recibos.to_csv('Recibos.csv', index=False, encoding='ANSI')